In [256]:
from flask import Flask, request, jsonify
from threading import Thread
import json
from datetime import datetime, timezone, timedelta
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
import time

In [257]:
def retrive_calendar_events(user, start, end):
    events_list = []
    token_path = "../Keys/"+user.split("@")[0]+".token"
    user_creds = Credentials.from_authorized_user_file("./Keys/userthree.amd.token")
    calendar_service = build("calendar", "v3", credentials=user_creds)
    events_result = calendar_service.events().list(calendarId='primary', timeMin=start,timeMax=end,singleEvents=True,orderBy='startTime').execute()
    events = events_result.get('items')
    
    for event in events : 
        attendee_list = []
        try:
            for attendee in event["attendees"]: 
                attendee_list.append(attendee['email'])
        except: 
            attendee_list.append("SELF")
        start_time = event["start"]["dateTime"]
        end_time = event["end"]["dateTime"]
        events_list.append(
            {"StartTime" : start_time, 
             "EndTime": end_time, 
             "NumAttendees" :len(set(attendee_list)), 
             "Attendees" : list(set(attendee_list)),
             "Summary" : event["summary"]})
    return events_list

retrive_calendar_events("userone.amd@gmail.com", "2025-07-17T10:30:00+05:30", "2025-07-17T11:00:00+05:30")

[]

In [259]:
app = Flask(__name__)
received_data = []

In [260]:
def generate_summary(email_content):
    url = "http://localhost:3000/v1/chat/completions"
    headers = {"Content-Type": "application/json"}
    data = {
        "model": "/home/user/Models/deepseek-ai/deepseek-llm-7b-chat",
        "messages": [
            {
                "role": "system",
                "content": "You are an AI scheduling assistant. Based on the following email content, generate a short summary or meeting title."
            },
            {
                "role": "user",
                "content": email_content
            }
        ],
        "stream": False,
        "max_tokens": 64
    }

    try:
        response = requests.post(url, headers=headers, json=data)
        result = response.json()
        return result["choices"][0]["message"]["content"].strip()
    except Exception as e:
        print(f"[LLM ERROR] Could not generate summary: {e}")
        return "Team Meeting"


In [177]:
def your_meeting_assistant(data):
    start_time = time.time()
    fields = ["Request_id", "Datetime", "Location", "From", "Subject", "EmailContent"]
    request_id, datetime_str, location, sender, subject, email_content = [data[k] for k in fields]
    attendees_list = [sender] + [a["email"] for a in data.get("Attendees", [])]

    event_start = "2025-07-17T10:30:00+05:30"
    event_end   = "2025-07-17T11:00:00+05:30"
    duration_mins = "30"

    output_attendees = []
    for email in attendees_list:
        events = retrive_calendar_events(email, event_start, event_end)
        events.append({"StartTime": event_start, "EndTime": event_end, "NumAttendees": len(attendees_list),
                       "Attendees": attendees_list, "Summary": subject})
        
        output_attendees.append({"email": email, "events": events})

    data.update({"Attendees": output_attendees, "EventStart": event_start, "EventEnd": event_end,
        "Duration_mins": duration_mins, "MetaData": {}})
    
    print(f"Processed in {time.time() - start_time:.3f} seconds")

    return data

In [270]:
IST_OFFSET_HOURS = 5
IST_OFFSET_MINUTES = 30
DEFAULT_MEETING_DURATION = 30
CALENDAR_LOOKUP_DAYS = 7
WORKING_HOURS_START = 9
WORKING_HOURS_END = 18

def your_meeting_assistant(request_json, base_url, keys_directory):
    client = OpenAI(api_key="NULL", base_url=base_url, timeout=None, max_retries=0)

    def parse_datetime(datetime_str):
        return datetime.fromisoformat(datetime_str)

    def retrive_calendar_events(user, start, end):
        events_list = []
        token_path = f"{keys_directory}/{user.split('@')[0]}.token"

        try:
            user_creds = Credentials.from_authorized_user_file(token_path)
            calendar_service = build("calendar", "v3", credentials=user_creds)
            events_result = calendar_service.events().list(
                calendarId='primary',
                timeMin=start,
                timeMax=end,
                singleEvents=True,
                orderBy='startTime'
            ).execute()
            events = events_result.get('items', [])

            for event in events:
                attendee_list = []
                for attendee in event.get("attendees", []):
                    attendee_list.append(attendee['email'])
                if not attendee_list:
                    attendee_list.append("SELF")
                start_time = event.get("start", {}).get("dateTime")
                end_time = event.get("end", {}).get("dateTime")
                if start_time and end_time:
                    events_list.append({
                        "StartTime": start_time,
                        "EndTime": end_time,
                        "NumAttendees": len(set(attendee_list)),
                        "Attendees": list(set(attendee_list)),
                        "Summary": event.get("summary", "No Title")
                    })
        except Exception as e:
            print(f"Error retrieving calendar for {user}: {e}")
            ist = timezone(timedelta(hours=IST_OFFSET_HOURS, minutes=IST_OFFSET_MINUTES))
            start_dt = parse_datetime(start)
            end_dt = parse_datetime(end)
            current_date = start_dt.date()
            while current_date <= end_dt.date():
                off_start = datetime.combine(current_date, datetime.min.time().replace(hour=WORKING_HOURS_END)).replace(tzinfo=ist)
                off_end = datetime.combine(current_date + timedelta(days=1), datetime.min.time().replace(hour=WORKING_HOURS_START)).replace(tzinfo=ist)
                events_list.append({
                    "StartTime": off_start.isoformat(),
                    "EndTime": off_end.isoformat(),
                    "NumAttendees": 1,
                    "Attendees": ["SELF"],
                    "Summary": "Off Hours"
                })
                current_date += timedelta(days=1)

        return events_list

    def parse_meeting_request(email_content, attendees_list):
        attendees_str = ", ".join([att["email"] for att in attendees_list])
        prompt = f"""
        You are an EVENT SCHEDULING EXPERT ASSISTANT. Parse the following meeting request and extract key information.

        Email Content: "{email_content}"
        Attendees: {attendees_str}

        Extract and return ONLY a JSON object with these fields:
        {{
            "duration_minutes": <meeting duration in minutes, default {DEFAULT_MEETING_DURATION} if not specified>,
            "time_preference": "<day preference like 'Thursday', 'next week', etc.>",
            "meeting_type": "<type of meeting from subject/content>",
            "urgency": "<high/medium/low based on content tone>"
        }}

        Rules:
        - If duration not specified, use {DEFAULT_MEETING_DURATION} minutes
        - Avoid SUNDAY and SATURDAY
        - Return ONLY valid JSON, no other text
        """

        try:
            response = client.chat.completions.create(
                model="deepseek-llm-7b-chat",
                temperature=0.0,
                messages=[{
                    "role": "user",
                    "content": prompt
                }]
            )
            content = response.choices[0].message.content.strip()
            if content.startswith("```json"):
                content = content.replace("```json", "").replace("```", "").strip()
            return json.loads(content)
        except Exception as e:
            print(f"Error parsing meeting request: {e}")
            return {
                "duration_minutes": DEFAULT_MEETING_DURATION,
                "time_preference": "any weekday",
                "meeting_type": "general meeting",
                "urgency": "medium"
            }

    def find_common_free_slots(all_users_events, date, duration_minutes):
        return [{
            "start": datetime.combine(date, datetime.min.time().replace(hour=10, minute=0)).replace(
                tzinfo=timezone(timedelta(hours=IST_OFFSET_HOURS, minutes=IST_OFFSET_MINUTES)))
        }]

    def opt_time(all_users_events, duration_minutes, time_preference):
        all_dates = set()
        for events in all_users_events.values():
            for event in events:
                start_date = parse_datetime(event['StartTime']).date()
                all_dates.add(start_date)
        date_range = sorted(list(all_dates))

        preferred_days = ["monday", "tuesday", "wednesday", "thursday", "friday"]
        filtered_dates = [d for d in date_range if d.strftime('%A').lower() in preferred_days]

        if "thursday" in time_preference.lower():
            filtered_dates = [d for d in filtered_dates if d.strftime('%A').lower() == "thursday"]
        elif "monday" in time_preference.lower():
            filtered_dates = [d for d in filtered_dates if d.strftime('%A').lower() == "monday"]

        for date in filtered_dates:
            free_slots = find_common_free_slots(all_users_events, date, duration_minutes)
            if free_slots:
                best_slot = free_slots[0]
                return {
                    'date': date,
                    'start_time': best_slot['start'],
                    'end_time': best_slot['start'] + timedelta(minutes=duration_minutes)
                }

        for date in date_range:
            free_slots = find_common_free_slots(all_users_events, date, duration_minutes)
            if free_slots:
                best_slot = free_slots[0]
                return {
                    'date': date,
                    'start_time': best_slot['start'],
                    'end_time': best_slot['start'] + timedelta(minutes=duration_minutes)
                }

        return None

    def generate_output_format(request_data, optimal_time, all_users_events, duration_minutes):
        if not optimal_time:
            raise Exception("No available time slots found")

        all_attendees = [request_data["From"]] + [att["email"] for att in request_data["Attendees"]]
        new_meeting = {
            "StartTime": optimal_time['start_time'].isoformat(),
            "EndTime": optimal_time['end_time'].isoformat(),
            "NumAttendees": len(all_attendees),
            "Attendees": all_attendees,
            "Summary": request_data["Subject"]
        }

        response = {
            "Request_id": request_data["Request_id"],
            "Datetime": request_data["Datetime"],
            "Location": request_data["Location"],
            "From": request_data["From"],
            "Attendees": [],
            "Subject": request_data["Subject"],
            "EmailContent": request_data["EmailContent"],
            "EventStart": optimal_time['start_time'].isoformat(),
            "EventEnd": optimal_time['end_time'].isoformat(),
            "Duration_mins": str(duration_minutes),
            "MetaData": {}
        }

        for attendee_email in all_attendees:
            attendee_events = all_users_events.get(attendee_email, []).copy()
            attendee_events.append(new_meeting)
            attendee_events.sort(key=lambda x: parse_datetime(x['StartTime']))
            response["Attendees"].append({
                "email": attendee_email,
                "events": attendee_events
            })

        return response

    request_data = json.loads(request_json) if isinstance(request_json, str) else request_json

    all_attendees = [request_data["From"]] + [att["email"] for att in request_data["Attendees"]]
    meeting_info = parse_meeting_request(request_data["EmailContent"], request_data["Attendees"])

    start_date = datetime.now(timezone(timedelta(hours=IST_OFFSET_HOURS, minutes=IST_OFFSET_MINUTES)))
    end_date = start_date + timedelta(days=CALENDAR_LOOKUP_DAYS)

    all_users_events = {}
    for attendee in all_attendees:
        all_users_events[attendee] = retrive_calendar_events(attendee, start_date.isoformat(), end_date.isoformat())

    time = opt_time(all_users_events, meeting_info["duration_minutes"], meeting_info["time_preference"])

    result = generate_output_format(request_data, time, all_users_events, meeting_info["duration_minutes"])
    return result

In [272]:
if __name__ == "__main__":
    input_json = {
        "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
        "Datetime": "09-07-2025T12:34:55",
        "Location": "IIT Mumbai",
        "From": "userone.amd@gmail.com",
        "Attendees": [
            {"email": "usertwo.amd@gmail.com"},
            {"email": "userthree.amd@gmail.com"}
        ],
        "Subject": "Agentic AI Project Status Update",
        "EmailContent": "Hi team, let's meet on Thursday for 30 minutes to discuss the status of Agentic AI Project."
    }

    # Set up your local values
    BASE_URL = f"http://localhost:3000/v1"  # vLLM API
    MODEL_PATH = "/home/user/Models/deepseek-ai/deepseek-llm-7b-chat"  # Your model's name (for local vLLM, must match)
    KEYS_DIRECTORY = "./Keys"

    output = your_meeting_assistant(
        input_json,
        base_url=BASE_URL,
        keys_directory=KEYS_DIRECTORY
    )

    print(json.dumps(output, indent=2))


Error parsing meeting request: Error code: 404 - {'object': 'error', 'message': 'The model `deepseek-llm-7b-chat` does not exist.', 'type': 'NotFoundError', 'param': None, 'code': 404}
{
  "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
  "Datetime": "09-07-2025T12:34:55",
  "Location": "IIT Mumbai",
  "From": "userone.amd@gmail.com",
  "Attendees": [
    {
      "email": "userone.amd@gmail.com",
      "events": [
        {
          "StartTime": "2025-07-13T18:00:00+05:30",
          "EndTime": "2025-07-14T09:00:00+05:30",
          "NumAttendees": 1,
          "Attendees": [
            "SELF"
          ],
          "Summary": "Off Hours"
        },
        {
          "StartTime": "2025-07-14T09:00:00+05:30",
          "EndTime": "2025-07-14T09:30:00+05:30",
          "NumAttendees": 3,
          "Attendees": [
            "usertwo.amd@gmail.com",
            "userthree.amd@gmail.com",
            "userone.amd@gmail.com"
          ],
          "Summary": "Agentic AI Project St

In [98]:
@app.route('/receive', methods=['POST'])
def receive():
    data = request.get_json()
    print(f"\n Received: {json.dumps(data, indent=2)}")
    new_data = your_meeting_assistant(data)  # Your AI Meeting Assistant Function Call
    received_data.append(data)
    print(f"\n\n\n Sending:\n {json.dumps(new_data, indent=2)}")
    return jsonify(new_data)

def run_flask():
    app.run(host='0.0.0.0', port=5000)

In [99]:
# Start Flask in a background thread
Thread(target=run_flask, daemon=True).start()

 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.
